# 1. Model

## 1.1 Language Modeling

Un modelo de lenguaje busca calcular la probabilidad de observar una secuancia de tokens o palabras $w_{0}, \ldots, w_{N}$, usando la regla de la cadena de probabilidades la probabilidad conjunta de observar tal secuencia se descompone en la siguiente expresión
:
\begin{equation*}
p(w_{0}, \ldots, w_{N}) = \prod_{i=0}^{N}p(w_{i}|\ldots, w_{i-1})
\end{equation*}


## 1.2 N-grams 
Un modelo de N-grams posee una fuerte suposición sobre las probabilidades condicionales de una palabra y su historia ($p(w_{i}|\ldots, w_{i-1})$), así la probabilidad de obserbar la palabra $w_{i}$ depende de los N-1 palabras anteriores. Por ejemplo:

### Unigrams
\begin{equation*}
p(w_{i}|\ldots, w_{i-1}) = p(w_{i})
\end{equation*}

### Bigrams
\begin{equation*}
p(w_{i}|\ldots, w_{i-1}) = p(w_{i}| w_{i-1})
\end{equation*}

### Trigrams
\begin{equation*}
p(w_{i}|\ldots, w_{i-1}) = p(w_{i}| w_{i-2}, w_{i-1})
\end{equation*}

## 1.3 Trigrams with linear interpolation

El baseline a implementar consiste de un modelo de trigramas con interpolación lineal para evitar el problema de **zero-count**, este problema ocurre cuando la probabilidad de observar un trigrama es cero, lo que trae por consecuencia que la probabilidad conjunta sea cero, pues usando regla de la cadena y markov de segundo orden la probabilidad conjunta se descompone en el producto de los trigramas de la secuencia y basta que un trigrama tenga probabilidad cero para hechar a perder el cálculo de la probabilidad.

### Trigrams
\begin{equation*}
p(w_{0}, \ldots, w_{N}) = \prod_{i=0}^{N} p(w_{i}| w_{i-2}, w_{i-1})
\end{equation*}

### Trigrams with linear interpolation

La probabilidad que entrega este modelo es la combinación convexa de las probabilidades que entregan los modelos unigrams, bigrams y trigrams.

\begin{align}
\nonumber
& p(w_{0}, \ldots, w_{N}) = \prod_{i=0}^{N}\lambda_{1} p(w_{i}| w_{i-2}, w_{i-1})+\lambda_{2} p(w_{i}| w_{i-1})+\lambda_{3} p(w_{i})\\ \nonumber
& \lambda_{i}\geq 0, \; \lambda_{1}+\lambda_{2}+\lambda_{3}=1\ \nonumber
\end{align}

Los parámetros del modelo se estiman contando casos favorables versus totales, por ejemplo la probabilidad estimada de un trigrama viene dada por la siguiente expresión:
\begin{equation*}
q(w_{i}|w_{i-2}, w_{i}) = \frac{Count(w_{i-2}, w_{i-1}, w_{i})}{Count(w_{i-2}, w_{i-1})}
\end{equation*}


# 2. Implementation 

In [1]:
import numpy as np
from nltk.corpus import reuters
from nltk import bigrams, trigrams
from collections import defaultdict

In [ ]:
class Trigrams():     
    def fit(self, corpus):
        """
        Ajusta el modelo en el corpus de entrenamiento, guarda los parámetros en atributos.

        Parameters:
            corpus: {list of str}, shape (corpus_size) 
            corpus de entrenamiento tokenizado.
        Returns:
            self: object
        """
        ## Crear diccionarios que guardaran la probabilidad de cada n-gram
        model1 = defaultdict(lambda: 0)
        model2 = defaultdict(lambda: defaultdict(lambda: 0))
        model3 = defaultdict(lambda: defaultdict(lambda: 0))

        ## Contar frecuencia de co-ocurrencia
        N = len(corpus)
        for sentence in corpus:
            # Unigrams
            model1[None]=N
            for w1 in sentence:
                model1[w1]+=1
            # Bigrams
            model2[None][None]=N
            for w1, w2 in bigrams(sentence, pad_right=True, pad_left=True):
                model2[w1][w2] += 1
            # Trigrams
            for w1, w2, w3 in trigrams(sentence, pad_right=True, pad_left=True):
                model3[(w1, w2)][w3] += 1

        ## Transformar conteo a probabilidades
        # Trigrams
        for w1_w2 in model3:
            w1, w2 = w1_w2[0], w1_w2[1]
            total_count = model2[w1][w2]
            for w3 in model3[w1_w2]:
                model3[w1_w2][w3] /= total_count
        # Bigrams        
        for w1 in model2:
            total_count = model1[w1]
            for w2 in model2[w1]:
                model2[w1][w2] /= total_count
        # Unigrams    
        total_count = sum(model1.values())
        for w1 in model1:
            model1[w1] /= total_count

        self.model1 = model1
        self.model2 = model2
        self.model3 = model3     
        
    def predict_proba_trigam(self, lamb, trigram):
        """
        Calcula la probabilidad de un trigrama usando interpolación lineal.
            - p(w3|w1,w2) = lamb_1*q(w3|w1,w2)+lamb_2*q(w3|w2)+lamb_3*q(w3)
            - lamb>=0, lamb_1+lamb_2+lamb_3=1
        Parameters:
            lamb: {array}, shape =3
            hiperparámetros del modelo, cada componente debe ser positivo y deben sumar 1.

        Returns:
            score: float
            probabilidad del trigrama
        """
        w1, w2, w3 = trigram
        proba = lamb[0]*self.model3[(w1,w2)][w3]+lamb[1]*self.model2[w2][w3]+lamb[2]*self.model1[w3]
        return proba
        
    def get_perplexity(self, lamb, corpus):

        """
        Obtiene la perplexity del modelo en un conjunto de validación
            -perplexity = exp(NLL/N), donde NLL es la negative-log-likelihood del modelo y 
             N el número de trigramas en el corpus.

        Parameters:
            - corpus: {list of str}, shape (corpus_size) 
              corpus de validación tokenizado.
            - lamb: {array}, shape =3
              hiperparámetros del modelo, cada componente debe ser positivo y deben sumar 1.
        Returns:
            score: float
            perplexity
        """

        N = 0 #contador de trigramas del conjunto de validación
        NLL = 0 #negative-log-likelihood
        for sentence in corpus:
            for trigram in trigrams(sentence, pad_right=True, pad_left=True):
                trigram_proba = self.predict_proba_trigam(lamb, trigram)
                NLL = NLL - np.log(trigram_proba) 
                N = N+1
        perplexity = np.exp(NLL/N)

        return perplexity
    

In [51]:
corpus = reuters.sents()
print(corpus[0])
print(len(corpus))

['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', '.', 'S', '.-', 'JAPAN', 'RIFT', 'Mounting', 'trade', 'friction', 'between', 'the', 'U', '.', 'S', '.', 'And', 'Japan', 'has', 'raised', 'fears', 'among', 'many', 'of', 'Asia', "'", 's', 'exporting', 'nations', 'that', 'the', 'row', 'could', 'inflict', 'far', '-', 'reaching', 'economic', 'damage', ',', 'businessmen', 'and', 'officials', 'said', '.']
54716


In [52]:
model = Trigrams()
%time model.fit(corpus)

Wall time: 11.7 s


In [53]:
lamb = [0.7, 0.2, 0.1]
%time model.get_perplexity(lamb, corpus)

Wall time: 11.9 s


8.383050732528654

In [58]:
#Calcular probabilidad de un trigrama
example = trigrams(corpus[0], pad_right=True, pad_left=True)
example_trigrams = []
for trigram in example:
    example_trigrams.append(trigram)
    
print(example_trigrams[2])
model.predict_proba_trigam(lamb, example_trigrams[2])

('ASIAN', 'EXPORTERS', 'FEAR')


0.7043479387228446